# A. Build First Model (no data preparation)

### Install packages / import namespaces

In [13]:
#r "nuget:Microsoft.ML"
using Microsoft.ML;
using Microsoft.ML.Data;

### Load data models from Models.cs file 

In [14]:
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Models.cs"

### Initialize the ML context (we need it for building the data and training pipelines)  

In [15]:
MLContext mlContext = new MLContext(0);

### Load data from csv file into a dataview

In [16]:
const string DATASET_PATH = "./sensors_data.csv";
IDataView data = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: DATASET_PATH,
    hasHeader: true,
    separatorChar: ',');

### Shuffle and split data to _trainingData_ and _testingData_ by a fraction of 0.2

In [17]:
var shuffledData = mlContext.Data.ShuffleRows(data, seed: 0);
var split = mlContext.Data.TrainTestSplit(shuffledData, testFraction: 0.2);
var trainingData = split.TrainSet;
var testingData = split.TestSet;

### Model builder pipeline

In [18]:
var featureColumns = new string[] { "Temperature", "Luminosity", "Infrared" };

var pipeline = mlContext.Transforms.Conversion.MapValueToKey("Label")
    .Append(mlContext.Transforms.Concatenate("Features", featureColumns))
    .Append(mlContext.Transforms.NormalizeMinMax("Features"))
    .Append(mlContext.MulticlassClassification.Trainers.SdcaNonCalibrated("Label", "Features"))
    .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

### Train the model

In [19]:
var model = pipeline.Fit(trainingData);

In [20]:
var predictions = model.Transform(testingData);
var metrics = mlContext.MulticlassClassification.Evaluate(predictions, "Label", "Score", "PredictedLabel");

In [21]:
#r "nuget:Microsoft.Data.Analysis"
using Microsoft.AspNetCore.Html;
using Microsoft.Data.Analysis;

#### Load formatter for metrics

In [22]:
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Formatters.cs"
var categories = new string[] { "FlashLight", "Infrared", "Day", "Lighter" };
Formatters.Load(categories);

In [23]:
metrics

EVALUATION: multi-class classification Class Value Note MacroAccuracy 0.968 the closer to 1, the better MicroAccuracy 0.960 the closer to 1, the better LogLoss 0.937 the closer to 0, the better LogLoss per Class FlashLight 1.877 the closer to 0, the better Infrared 0.000 the closer to 0, the better Day 0.000 the closer to 0, the better Lighter 1.309 the closer to 0, the better

In [24]:
var sampleData = new ModelInput
{
    Luminosity = 92F,
    Temperature = 122F,
    Infrared = 0F
};

var predictor = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(model);
var predicted = predictor.Predict(sampleData);

display(p[style: "font-size: 24px"]($"{predicted.PredictedLabel}"));

FlashLight